# Check on the accuracy of the calculation of the QOCT gradients

In [ ]:
try:
    get_ipython
    isnotebook = True
except:
    isnotebook = False

import os
import sys
import numpy as np
import scipy as sp
import matplotlib
if not isnotebook:
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
import time
import qutip as qt
from qutip_qip.operations import toffoli

In [ ]:
import qocttools
import qocttools.models.GdW30 as GdW30
import qocttools.hamiltonians as hamiltonians
import qocttools.math_extra as math_extra
import qocttools.pulses as pulses
import qocttools.qoct as qoct
import qocttools.solvers as solvers
import qocttools.target as target

In [ ]:
qocttools.about()

In [ ]:
data = []

# Introduction

In order to calculate the optimal pulse that induces a given reaction in a quantum system, one defines a function of that pulse that must be optimized. One important ingredient for the optimization is derivative of this function with respect to the control parameters that define the pulse. In this script we check that this gradient or derivative is calculated correctly.

# Model

The model is defined by the Hamiltonian:

\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
where the time-independent part is given by:
\begin{equation}
        \hat{H}_0 = D\bigg[\hat{S}_z^2 - \frac{1}{3}S(S + 1)\bigg] + E[\hat{S}_x^2 - \hat{S}_y^2] - g\mu_B\hat{\vec{S}}\cdot\vec{H}
\end{equation}
and the time-dependent part is:
\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
The perturbation is a magnetic field:
\begin{equation}
        \hat{V} = -g\mu_B\hat{\vec{S}}\cdot\vec{H}_m 
\end{equation}

In this case:

* $S = 7/2$

* $D$ = 1281 MHz

* $E$ = 294 MHz

* $\vec{H} = (0.15, 0.0, 0.0)$ T

* $\vec{H}_m = (0, 0.001, 0.0)$ T

In [ ]:
S = 7/2 # spin
E = 294 # value in MHz
D = 1281 # value in MHz
dim = int(2*S + 1) #matrix dim

target_level = 1

In [ ]:
H = np.array([0.15, 0, 0.0], dtype = float) #magnetic field in T
H_m = np.array([0, 0.001, 0], dtype = float) #only in presence of perturbation (T)
H0 = GdW30.hGdW30(D, E, H)
V = GdW30.vGdW30(H_m)

In [ ]:
eigenvalues, eigenstates = H0.eigenstates()
#qt.fileio.qsave(eigenstates, "eigenstates")
eigenstates = qt.fileio.qload("eigenstates")

In [ ]:
# In principle, we could just transform V with the recently obtained eigenstates. Unfortunately, that
# would make the test results different in different computers, as the eigenstates can have different phases.
H0 = H0.transform(eigenstates) - eigenvalues[0]
V = V.transform(eigenstates)

In [ ]:
H = hamiltonians.hamiltonian(H0, [V])

In [ ]:
w = np.zeros(dim-1)
taui = np.zeros(dim-1)
for i in range(dim-1):
    w[i] = eigenvalues[i+1] - eigenvalues[i]
    taui[i] = 2.0*np.pi/w[i]
    print("Transition {:d}: w = {:f} MHz, tau = {:f} ns".format(i, w[i], 1000.0*taui[i]/(2.0*np.pi)))

# Time array definition

In [ ]:
T = 5*taui[0]
print("T = {:f} us*2*pi = {:f} ns".format(T, 1000*T/(2.0*np.pi)))
time = math_extra.timegrid(H0, T, 0.5)
print('# Time steps =', time.shape[0])
print('# Delta t =', time[1])

# Control function

The control function is parametrized with the Fourier expansion as follow:
\begin{equation}
    f(u, t) = \frac{1}{\sqrt{T}}u_0 + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k}\cos(\omega_kt) + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k + 1}\sin(\omega_kt),
\end{equation}
where $u_0\dots u_{2M + 1}$ are the control parameters. This way, we can compute the derivate respect any control parameter as
\begin{equation}
    \frac{\partial f}{\partial u_m}(u, t) = f(e_m, t),
\end{equation}
where $e_m$ is the set of parameters where all of them are zero except the m-th ane, that is equal to one.

This pulse parametrization is included in the typical_pulses.py file as pulse class.

In [ ]:
M = 10

omega = np.zeros(M+1)
omega[0] = 0.0
for k in range(1, M+1):
    omega[k] = (2.0*np.pi/T) * k
    print("omega[{:d}] = {:f} MHz".format(k, omega[k]))

# u = 1.0*np.random.rand((2*M + 1))
u = np.zeros(2*M+1)
u[2] = 1.0
u[3] = 1.0
f = pulses.pulse("fourier", T, u = u)

print("Amplitudes:")
print("{:f} mT".format(u[0]/np.sqrt(T)))
for m in range(1, 2*M+1):
    print("{:f} mT".format(2.0*u[m]/(np.sqrt(T))))

In [ ]:
fig, ax = plt.subplots()

ax.plot(time * 1000/(2.0*np.pi), f.fu(time, u))
ax.set_xlabel("Time (ns)")
ax.set_xlim(left = 0.0, right = time[-1]*1000/(2.0*np.pi))
ax.set_ylabel("f(t) (mT)")
if isnotebook:
    plt.show()
else:
    fig.savefig("pulse.pdf")

# QOCT equations

Given the initial state of the system we know that:
\begin{equation}
    i\frac{d}{dt}U(t) = [\hat{H}_0 + f(u, t)\hat{V}]U(t)
\end{equation}
\begin{equation}
    U(0) = I
\end{equation}

if we want to induce a transition from the initial state to another one first we need to define a function to optimize such that it minimizes when the control parameters generate the pulse that induce the transition. In this case, this function is defined as
\begin{equation}
    G(u) = F(U[u], u) = J_1(U[u](T)))
\end{equation}
where
\begin{equation}
    J_1(c) = |U_{target}\cdot U|^2
\end{equation}
This way, the gradient of $G(u)$ is given by
\begin{equation}
    \frac{\partial G}{\partial u_m}(u) = 2\text{Im}\int_o^Tdt\frac{\partial f}{\partial u_m}(u, t)B^\dagger[u](t)\cdot(\hat{V}U[u](t))
\end{equation}
Where $d[u](t)$ is called "costate" and can be obtained as following:
\begin{equation}
    i\frac{d}{dt}B(t) = \hat{H}^\dagger(u, t)B[u](t)
\end{equation}
\begin{equation}
    B[u](T) = (U_{target}\cdot U[u](T))U_{target}
\end{equation}

# QOCT target function definition

In [ ]:
U_target = qt.Qobj(toffoli().full())
U_target_int = (1j*T*H0).expm() * \
               qt.Qobj(toffoli().full())

U_0 = qt.qeye(dim) #initial state

# Comparison, Schrödinger picture

In [ ]:
tg = target.Target('evolutionoperator', Utarget = U_target)

In [ ]:
opt = qoct.Qoct(H, T, time.shape[0], tg, f, U_0,
                interaction_picture = False, solve_method = 'cfmagnus4')

In [ ]:
derqoct, dernum, error, elapsed_time = opt.check_grad(u, 2)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))
data.append(derqoct)

# Comparison, interaction picture

In [ ]:
tg = target.Target('evolutionoperator', Utarget = U_target_int)

In [ ]:
opt = qoct.Qoct(H, T, time.shape[0], tg, f, U_0,
                interaction_picture = True, solve_method = 'cfmagnus4')

In [ ]:
derqoct, dernum, error, elapsed_time = opt.check_grad(u, 2)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))
data.append(derqoct)

# Output data

In [ ]:
with open("data", "w") as datafile:
    for i in data:
        datafile.write("{:.14e}\n".format(i))